## Relational Synthetic Data Generation

This ML solution generates relational synthetic data that preserves properties of original tables and their relationships with privacy.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: clean-sentiment-classification-labels. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure dataset](#B.-Configure-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Execute optimization model](#3.-Execute-optimization-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Execute model](#3.2-Execute-model)
    1. [Visualize Output](#3.3-Inspect-the-Output-in-S3)
1. [Clean-up](#4.-Clean-up)
	1. [Unsubscribe to the listing (optional)](#Unsubscribe-to-the-listing-(optional))

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page **Relational Synthetic Data Generation**
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
algo_arn = "arn:aws:sagemaker:us-east-2:786796469737:algorithm/relational-synthetic-structural-data-v1"

### 2. Prepare dataset

In [2]:
import os
import json 
import uuid
import boto3
import pickle
import base64
import tarfile
from pprint import pprint

import numpy as np
import pandas as pd

import urllib.request
from urllib.parse import urlparse

import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role

#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as described:
* You need "input.zip" which contains parent_table_input.csv, child_table_input.csv, parent_table_variable_info.json, child_table_variable_info.json, input_parameters.json, relational_table_structure.json 
* parent_table_input.csv, child_table_input.csv are the original relational datasets
* parent_table_variable_info.json & child_table_variable_info.json should contain the variable type (ID, name, categorical, numerical) and variable sensitivity (True, False)
*  input_parameters.json should contain the parent_table_drop_cols, child_table_drop_cols and scale factor (no. of rows in synthetic compared to original)
*  relational_table_structure.json contains the relationship info between the tables
* For detailed instructions, please refer sample notebook and algorithm input details

#### B. Configure dataset

In [3]:
training_dataset="Input/input.zip"

#### C. Upload datasets to Amazon S3

In [4]:
role = get_execution_role()

In [ ]:
sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
# training input location
common_prefix = "relational-synthetic-data-generator"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "Input" #Input directory in Jupyter Server
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix) #uploads data from jupyter server to S3
print("Training input uploaded to " + training_input)

## 3. Execute the training process

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to execute a training pipeline to get clean sentiment class labels using clean-sentiment-classification-labels Algorithm. 

### 3.1 Set up environment

In [7]:
output_location = 's3://{}/relational-synthetic-data-generator/{}'.format(bucket, 'output')

### 3.2 Execute model

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [8]:
training_instance_type="ml.m5.4xlarge"

In [9]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="relational-synthetic-data-generator",
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)

#Run the training job.
estimator.fit({"training": training_input})

2023-01-09 10:44:41 Starting - Starting the training job...
2023-01-09 10:45:05 Starting - Preparing the instances for trainingProfilerReport-1673261081: InProgress
......
2023-01-09 10:46:09 Downloading - Downloading input data
2023-01-09 10:46:09 Training - Downloading the training image.........
2023-01-09 10:47:25 Training - Training image download completed. Training in progress.....The table lineage is from left to right in the list: 
#015  0%|          | 0/1000 [00:00<?, ?it/s]#015Sampling conditions:   0%|          | 0/1000 [00:00<?, ?it/s]#015Sampling conditions:   0%|          | 4/1000 [00:00<00:26, 38.04it/s]#015Sampling conditions:   1%|          | 8/1000 [00:00<00:25, 39.07it/s]#015Sampling conditions:   1%|          | 12/1000 [00:00<00:25, 39.37it/s]#015Sampling conditions:   2%|▏         | 16/1000 [00:00<00:24, 39.59it/s]#015Sampling conditions:   2%|▏         | 20/1000 [00:00<00:24, 39.68it/s]#015Sampling conditions:   2%|▏         | 24/1000 [00:00<00:24, 39.75it/s]#015

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

In [ ]:
#output is available on following path
estimator.output_path

## Note: Inferencing is done within training pipeline. Real time inference endpoint/batch transform job is not required.

### 3.3 Inspect the Output in S3

In [11]:
parsed_url = urlparse(estimator.output_path)
bucket_name = parsed_url.netloc
file_key = parsed_url.path[1:]+'/'+estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

s3_client = sagemaker_session.boto_session.client('s3')
response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key)

In [12]:
bucketFolder = estimator.output_path.rsplit('/')[3] +'/output/'+ estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

In [13]:
s3_conn = boto3.client("s3")
bucket_name=bucket
with open('output.tar.gz', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder, f)
    print("Output file loaded from bucket")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Output file loaded from bucket


In [14]:
with tarfile.open('output.tar.gz') as file:
    file.extractall('./output')

In [23]:
#synthetic_datasets
print("Synthetic Datasets read successfully")
parent_table = pd.read_csv(os.path.join("output", "parent_table.csv"), index_col=0)
child_table = pd.read_csv(os.path.join("output", "child_table.csv"), index_col=0)

print('Performance metrics read successfully')
eval_parent_table = pd.read_csv(os.path.join("output", "eval_parent_table.csv"), index_col=0)
eval_child_table = pd.read_csv(os.path.join("output", "eval_child_table.csv"), index_col=0)

Synthetic Datasets read successfully
Performance metrics read successfully


**Synthethic output data**

In [24]:
parent_table.head()

,user_id,name,age,city,state,country,zip,latitude,longitude,gender,traffic_source
0,1,RANDOLPH CONNELL,32,Phoenix,Arizona,USA,85001,32.233211,-111.559751,Male,Email
1,2,JACK HOLT,77,Phoenix,Arizona,USA,85029,32.976531,-112.713756,Male,Display
2,3,RANDOLPH BARRY,58,Tucson,Arizona,USA,85007,32.455123,-112.094229,Male,Search
3,4,STEVEN GOSSETT,81,Phoenix,Arizona,USA,85006,32.731460,-113.081407,Male,Search
4,5,BETTYE HALL,15,Phoenix,Arizona,USA,85016,32.168397,-111.532285,Male,Organic


In [25]:
child_table.head()

,order_id,user_id,sale_price,status
0,1,1,82.333152,Cancelled
1,2,2,23.886154,Cancelled
2,3,3,31.302740,Complete
3,4,4,42.936255,Cancelled
4,5,5,12.264545,Cancelled


**Performance Metrics**

In [26]:
eval_parent_table.head()

,Synthesizer,Discrete Cols Similarity (CSTest),Continuous Cols Similarity (KSTest),All Cols Similarity (evaluation_score),Privacy at Risk (Exact)
0,SDV,0.95802,0.751,85.451025,23.8


In [27]:
eval_child_table.head()

,Synthesizer,Discrete Cols Similarity (CSTest),Continuous Cols Similarity (KSTest),All Cols Similarity (evaluation_score),Privacy at Risk (Exact)
0,SDV,0.999377,0.798,89.868826,31.0


#### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.